In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py

import lightgbm as lgb 

from sklearn import preprocessing 

## NN based approach for electron energy regression

A neural network is made with X structure from the keras framework

What to implement 
* Cross-Validation
* Hyperparameter optimisation

# Data imports

In [2]:
def load_data(name):
    with h5py.File(f'{name}.h5', 'r') as f:
        filename = name.split('/')[1]
        return pd.DataFrame(f[filename][:], dtype=np.float64)

train = load_data('data/train')
test  = load_data('data/test')

# Select only true electrons for energy regression
train_mask = train['Truth'][:130000] == 1
validate_mask = train['Truth'][130000:] == 1

#scaler_train = preprocessing.StandardScaler().fit(train)
#scaler_test = preprocessing.StandardScaler().fit(test)
#train = pd.DataFrame(scaler_train.transform(train), columns = train.columns)
#test = pd.DataFrame(scaler_test.transform(test), columns = test.columns)

# Variable list from https://www.nbi.dk/~petersen/Teaching/ML2023/InitialProject/VariableList.html
all_variables = ['actualInteractionsPerCrossing', 'averageInteractionsPerCrossing', 'correctedActualMu', 'correctedAverageMu', 'correctedScaledActualMu', 'correctedScaledAverageMu', 'NvtxReco', 'p_nTracks', 'p_pt_track', 'p_eta', 'p_phi', 'p_charge', 'p_qOverP', 'p_z0', 'p_d0', 'p_sigmad0', 'p_d0Sig', 'p_EptRatio', 'p_dPOverP', 'p_z0theta', 'p_etaCluster', 'p_phiCluster', 'p_eCluster', 'p_rawEtaCluster', 'p_rawPhiCluster', 'p_rawECluster', 'p_eClusterLr0', 'p_eClusterLr1', 'p_eClusterLr2', 'p_eClusterLr3', 'p_etaClusterLr1', 'p_etaClusterLr2', 'p_phiClusterLr2', 'p_eAccCluster', 'p_f0Cluster', 'p_etaCalo', 'p_phiCalo', 'p_eTileGap3Cluster', 'p_cellIndexCluster', 'p_phiModCalo', 'p_etaModCalo', 'p_dPhiTH3', 'p_R12', 'p_fTG3', 'p_weta2', 'p_Reta', 'p_Rphi', 'p_Eratio', 'p_f1', 'p_f3', 'p_Rhad', 'p_Rhad1', 'p_deltaEta1', 'p_deltaPhiRescaled2', 'p_TRTPID', 'p_TRTTrackOccupancy', 'p_numberOfInnermostPixelHits', 'p_numberOfPixelHits', 'p_numberOfSCTHits', 'p_numberOfTRTHits', 'p_numberOfTRTXenonHits', 'p_chi2', 'p_ndof', 'p_SharedMuonTrack', 'p_E7x7_Lr2', 'p_E7x7_Lr3', 'p_E_Lr0_HiG', 'p_E_Lr0_LowG', 'p_E_Lr0_MedG', 'p_E_Lr1_HiG', 'p_E_Lr1_LowG', 'p_E_Lr1_MedG', 'p_E_Lr2_HiG', 'p_E_Lr2_LowG', 'p_E_Lr2_MedG', 'p_E_Lr3_HiG', 'p_E_Lr3_LowG', 'p_E_Lr3_MedG', 'p_ambiguityType', 'p_asy1', 'p_author', 'p_barys1', 'p_core57cellsEnergyCorrection', 'p_deltaEta0', 'p_deltaEta2', 'p_deltaEta3', 'p_deltaPhi0', 'p_deltaPhi1', 'p_deltaPhi2', 'p_deltaPhi3', 'p_deltaPhiFromLastMeasurement', 'p_deltaPhiRescaled0', 'p_deltaPhiRescaled1', 'p_deltaPhiRescaled3', 'p_e1152', 'p_e132', 'p_e235', 'p_e255', 'p_e2ts1', 'p_ecore', 'p_emins1', 'p_etconeCorrBitset', 'p_ethad', 'p_ethad1', 'p_f1core', 'p_f3core', 'p_maxEcell_energy', 'p_maxEcell_gain', 'p_maxEcell_time', 'p_maxEcell_x', 'p_maxEcell_y', 'p_maxEcell_z', 'p_nCells_Lr0_HiG', 'p_nCells_Lr0_LowG', 'p_nCells_Lr0_MedG', 'p_nCells_Lr1_HiG', 'p_nCells_Lr1_LowG', 'p_nCells_Lr1_MedG', 'p_nCells_Lr2_HiG', 'p_nCells_Lr2_LowG', 'p_nCells_Lr2_MedG', 'p_nCells_Lr3_HiG', 'p_nCells_Lr3_LowG', 'p_nCells_Lr3_MedG', 'p_pos', 'p_pos7', 'p_poscs1', 'p_poscs2', 'p_ptconeCorrBitset', 'p_ptconecoreTrackPtrCorrection', 'p_r33over37allcalo', 'p_topoetconeCorrBitset', 'p_topoetconecoreConeEnergyCorrection', 'p_topoetconecoreConeSCEnergyCorrection', 'p_weta1', 'p_widths1', 'p_widths2', 'p_wtots1', 'p_e233', 'p_e237', 'p_e277', 'p_e2tsts1', 'p_ehad1', 'p_emaxs1', 'p_fracs1', 'p_DeltaE', 'p_E3x5_Lr0', 'p_E3x5_Lr1', 'p_E3x5_Lr2', 'p_E3x5_Lr3', 'p_E5x7_Lr0', 'p_E5x7_Lr1', 'p_E5x7_Lr2', 'p_E5x7_Lr3', 'p_E7x11_Lr0', 'p_E7x11_Lr1', 'p_E7x11_Lr2', 'p_E7x11_Lr3', 'p_E7x7_Lr0', 'p_E7x7_Lr1' ]


X_train = train[all_variables][:130000]
y_train = train['p_truth_E'][:130000]
X_validate = train[all_variables][130000:]
y_validate = train['p_truth_E'][130000:]
X_test = test[all_variables]

with open('Classification_JuliusFoverskov_XGBoost_VariableList.txt','r') as f:
    sorted_variables = [item.strip() for item in f]

# Only pick 15 most important features for clustering, importances optained from permutation importance
sorted_variables = sorted_variables[:15]

X_train = X_train[sorted_variables]
X_validate = X_validate[sorted_variables]



In [3]:


data_train = lgb.Dataset(X_train, label=y_train)
params = {'objective': 'regression',
          'boosting_type': 'gbdt',
          'metric': 'mean_squared_error',
          'learning_rate': 0.01,
          'num_leaves': 10,
          'max_depth': 10,
          'min_data': 10,
          'verbose': 0,
          'force_col_wise': True}

    
    

lgb_clf = lgb.train(params, train_set=data_train, num_boost_round=300)
y_pred = np.around(lgb_clf.predict(X_train.values))

In [4]:

from sklearn.metrics import mean_squared_error

# Assuming you have trained a LightGBM model and have the predictions and ground truth
y_pred = lgb_clf.predict(X_validate)  # Replace X_test with your test data
mse = mean_squared_error(y_validate, y_pred)

print("Mean Squared Error (MSE):", mse)

preds = lgb_clf.predict(X_test[sorted_variables])
preds

Mean Squared Error (MSE): 1780767430.2134445


array([158135.40980146, 121317.87191535, 154378.98803863, ...,
       101112.45879904,  44745.62876193, 183020.86905364])

In [5]:
# Write results to a file

# Probabilities of particle being an electron
with open('Regression_JuliusFoverskov_LightGBM.txt','w') as f:
    for index, prediction in zip(X_test.index, preds):
        f.write(f"{index}, {prediction}\n")

# List of most important features from permutation importance for electron classification
with open('Regression_JuliusFoverskov_LightGBM_VariableList.txt','w') as f:
    f.write("\n".join(map(str, sorted_variables)))